In [1]:
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork as BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from pgmpy.inference import VariableElimination


In [5]:
df = pd.read_csv('heart.csv')
print(df.head())

le = LabelEncoder()
for column in df.columns:
  if df[column].dtype == 'object':
    df[column] = le.fit_transform(df[column])

model = BayesianModel([('age', 'trestbps'), ('age', 'fbs'), ('sex', 'trestbps'), ('exang', 'trestbps'), ('trestbps', 'target')])
model.fit(df, estimator=MaximumLikelihoodEstimator)

model.fit(df, estimator = MaximumLikelihoodEstimator)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'sex': 'N', 'cp': 'N', 'trestbps': 'N', 'chol': 'N', 'fbs': 'N', 'restecg': 'N', 'thalach': 'N', 'exang': 'N', 'oldpeak': 'N', 'slope': 'N', 'ca': 'N', 'thal': 'N', 'target': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'sex': 'N', 'cp': 'N', 'trestbps': 'N', 'chol': 'N', 'fbs': 'N', 'restecg': 'N', 'thalach': 'N', 'exang': 'N', 'oldpeak': 'N', 'slope': 'N', 'ca': 'N', 'thal': 'N', 'target': 'N'}


   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  


In [7]:
test_cases = [
    {'age': 76, 'sex': 1, 'trestbps': 170, 'fbs': 1, 'exang': 1},  # Very high BP, old age, high blood sugar, angina
    {'age': 54, 'sex': 0, 'trestbps': 130, 'fbs': 0, 'exang': 0},  # Normal case, female, good vitals
    {'age': 63, 'sex': 1, 'trestbps': 145, 'fbs': 1, 'exang': 1},  # Older male, high sugar, angina
    {'age': 45, 'sex': 0, 'trestbps': 120, 'fbs': 0, 'exang': 0},  # Healthy young female
    {'age': 58, 'sex': 1, 'trestbps': 150, 'fbs': 1, 'exang': 0},  # Prehypertension + high sugar
    {'age': 67, 'sex': 0, 'trestbps': 180, 'fbs': 1, 'exang': 1},  # Elderly female with multiple risk factors
    {'age': 39, 'sex': 1, 'trestbps': 110, 'fbs': 0, 'exang': 0},  # Young healthy male
    {'age': 70, 'sex': 1, 'trestbps': 160, 'fbs': 1, 'exang': 1},  # High BP and high sugar + exercise angina
    {'age': 50, 'sex': 0, 'trestbps': 140, 'fbs': 0, 'exang': 0},  # Moderate BP, normal sugar
    {'age': 60, 'sex': 1, 'trestbps': 135, 'fbs': 1, 'exang': 0}   # Older male, high sugar, no angina
]


infer = VariableElimination(model)
for case in test_cases:
    result = infer.query(variables=['target'], evidence=case)
    print(f"Prediction for {case}:")
    print(result)
    print("\n")


Prediction for {'age': 76, 'sex': 1, 'trestbps': 170, 'fbs': 1, 'exang': 1}:
+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.7333 |
+-----------+---------------+
| target(1) |        0.2667 |
+-----------+---------------+


Prediction for {'age': 54, 'sex': 0, 'trestbps': 130, 'fbs': 0, 'exang': 0}:
+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.3902 |
+-----------+---------------+
| target(1) |        0.6098 |
+-----------+---------------+


Prediction for {'age': 63, 'sex': 1, 'trestbps': 145, 'fbs': 1, 'exang': 1}:
+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.8235 |
+-----------+---------------+
| target(1) |        0.1765 |
+-----------+---------------+


Prediction for {'age': 45, 'sex': 0, 'trestbps': 120, 'fbs': 0, 'exang': 0}:
+-----------+---------------+
| target    |   phi(target